In [1]:
import numpy as np
import pandas as pd

from src.util import Const

## Load DB data

In [2]:
horizon = pd.read_csv(
    Const.HORIZON_DB_FILE_PATH,
    usecols=['cokey', 'om_r'],
)
horizon.rename(
    columns={
        'cokey': Const.DB_COMP_ID,
        'om_r': Const.DB_OM,
    },
    inplace=True,
)

horizon[Const.DB_COMP_ID] = horizon[Const.DB_COMP_ID].astype(np.uint32)
horizon[Const.DB_OM] = horizon[Const.DB_OM].astype(np.float32)
horizon[Const.DB_OM] = horizon[Const.DB_OM] / 100
horizon.set_index(Const.DB_COMP_ID, inplace=True)
horizon

,Organic Matter concentration
Component ID,
14175226,0.0300
14175226,NaN
14175226,NaN
14175227,0.0125
14175227,NaN
...,...
14263675,0.0025
14263675,NaN
14263676,0.0025


In [3]:
component = pd.read_csv(
    Const.COMPONENT_DB_FILE_PATH,
    usecols=['cokey', 'mukey'],
)
component.rename(
    columns={
        'cokey': Const.DB_COMP_ID,
        'mukey': Const.DB_MU_ID,
    },
    inplace=True,
)

component[Const.DB_COMP_ID] = component[Const.DB_COMP_ID].astype(np.uint32)
component[Const.DB_MU_ID] = component[Const.DB_MU_ID].astype(np.uint32)
component.set_index(Const.DB_COMP_ID, inplace=True)
component

,Map Unit ID
Component ID,
14175226,657753
14175227,657753
14175228,657753
14175229,657753
14175230,657753
...,...
14263672,675172
14263673,675172
14263674,675172


## Clean DB data

In [4]:
initial_size = len(horizon)
print(f'Initial Horizon size = {initial_size}')

horizon.dropna(inplace=True)
nan_size = initial_size - len(horizon)
print(f'\tNaN values removed from Horizon = {nan_size}')

horizon.replace('', np.nan, inplace=True)
horizon.dropna(inplace=True)
empty_size = initial_size - nan_size - len(horizon)
print(f'\tEmpty values removed from Horizon = {empty_size}')

print(f'Final Horizon size = {len(horizon)}')

Initial Horizon size = 9577
	NaN values removed from Horizon = 4226
	Invalid organic matter percentage values removed from Horizon = 0
	Empty values removed from Horizon = 0
Final Horizon size = 5351


In [5]:
horizon

,Organic Matter concentration
Component ID,
14175226,0.0300
14175227,0.0125
14175228,0.0200
14175229,0.0125
14175229,0.0025
...,...
14263675,0.0150
14263675,0.0025
14263676,0.0025


In [6]:
initial_size = len(component)
print(f'Initial Component size = {initial_size}')

component.dropna(inplace=True)
nan_size = initial_size - len(component)
print(f'\tNaN values removed from Component = {nan_size}')

component.replace('', np.nan, inplace=True)
component.dropna(inplace=True)
empty_size = initial_size - nan_size - len(component)
print(f'\tEmpty values removed from Component = {empty_size}')

print(f'Final Component size = {len(component)}')

del nan_size, empty_size

Initial Component size = 2945
	NaN values removed from Component = 0
	Empty values removed from Component = 0
Final Component size = 2945


In [7]:
component

,Map Unit ID
Component ID,
14175226,657753
14175227,657753
14175228,657753
14175229,657753
14175230,657753
...,...
14263672,675172
14263673,675172
14263674,675172


## Intersect tables

In [8]:
data = component.join(
    horizon,
    how='inner',
).copy()

del horizon, component

In [9]:
data

,Map Unit ID,Organic Matter concentration
Component ID,,
14175226,657753,0.0300
14175227,657753,0.0125
14175228,657753,0.0200
14175229,657753,0.0125
14175229,657753,0.0025
...,...,...
14263675,675172,0.0150
14263675,675172,0.0025
14263676,675172,0.0025


### Keep duplicated `MapUnit`/`Component` with highest `OrganicMatter` value

In [10]:
initial_size = len(data)

data.sort_values(
    by=[Const.DB_MU_ID, Const.DB_OM],
    ascending=False,
    inplace=True,
)
data.drop_duplicates(subset=[Const.DB_MU_ID], keep='first', inplace=True)

print(f'Unnecessary Organic matter values removed = {initial_size - len(data)}')

data.reset_index(drop=True, inplace=True)
del initial_size

Unnecessary Organic matter values removed = 5096


In [11]:
data

,Map Unit ID,Organic Matter concentration
0,675172,0.045
1,675154,0.030
2,675138,0.030
3,669773,0.500
4,669761,0.150
...,...,...
250,657757,0.015
251,657756,0.020
252,657755,0.020
253,657754,0.030
